# Sentinel2 data selection, preprocessing, download

### Import libraries

In [5]:
import geemap
import ee
import geopandas as gpd
import folium
import json
import shapely.geometry
import shapely.ops
import datetime
from IPython.display import display

Authentication:

In [6]:
ee.Authenticate()

True

In [7]:
ee.Initialize()

### Add a basemap

In [8]:
# Path to the shapefile
shapefile_path = r'C:\Users\andre\OneDrive - Politecnico di Milano\Earth_observation\Practices-20240418\EO practices - module A\P1_Introduction to data and tools for multispectral satellite imagery\shp\shanghai_frame_s2.shp'

try:
    # Load the shapefile into a GeoDataFrame
    gdf = gpd.read_file(shapefile_path)
    
    # Reproject the GeoDataFrame to EPSG:4326 (WGS84) if not already in that CRS
    if gdf.crs != 'EPSG:4326':
        gdf = gdf.to_crs(epsg=4326)
    
    # Initialize the Folium map centered around the centroid of the shapefile
    centroid = gdf.geometry.centroid.iloc[0]  # Taking the centroid of the first feature
    m = folium.Map(location=[centroid.y, centroid.x], zoom_start=9)
    
    # Add the GeoDataFrame as a GeoJSON layer to the map
    folium.GeoJson(gdf).add_to(m)
    
    # Display the map
    display(m)
    
except Exception as e:
    print(f"Error reading shapefile '{shapefile_path}': {e}")

C:\Users\andre\AppData\Local\Temp\ipykernel_15660\2826157215.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid.iloc[0]  # Taking the centroid of the first feature


### Understanding the Sentinel products avalability from the selected frame

In [9]:
# Extract the first geometry from the GeoDataFrame 
geometry = gdf.iloc[0].geometry

# Ensure the exterior ring is oriented counterclockwise (very important step otherwise here stuck)
# https://github.com/chrieke/geojson-invalid-geometry
if not geometry.exterior.is_ccw:
    geometry = shapely.geometry.polygon.orient(geometry, sign=1.0)
    print("flipped")
    
# Remove Z values from the geometry coordinates
def remove_z(geometry):
    if isinstance(geometry, shapely.geometry.Polygon):
        return shapely.geometry.Polygon(
            [(x, y) for x, y, *_ in geometry.exterior.coords],
            [shapely.geometry.LinearRing([(x, y) for x, y, *_ in ring.coords]) for ring in geometry.interiors]
        )
    return geometry

geometry = remove_z(geometry)

# Convert the geometry to GeoJSON format
roi_geojson = shapely.geometry.mapping(geometry)

# Validate GeoJSON structure by printing
print("GeoJSON Structure:")
print(json.dumps(roi_geojson, indent=2))

# Try to create an ee.Geometry object and catch any exceptions
try:
    ee_geometry = ee.Geometry(roi_geojson)
    print("ee.Geometry created successfully.")
except Exception as e:
    print(f"Error creating ee.Geometry: {e}")
    
# Define the Sentinel-2 image collection
try:
    s2_collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
        .filterBounds(ee_geometry) \
        .filterDate('2010-01-01', '2021-12-31') \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .filter(ee.Filter.Or(ee.Filter.stringContains('system:index', 'T51RUQ'),
                             ee.Filter.stringContains('system:index', 'T51RVQ'))) \
        .sort('CLOUDY_PIXEL_PERCENTAGE') \
        .sort('system:time_start', True)

    # Get the list of available Sentinel-2 products
    products_available = s2_collection.aggregate_array('system:index').getInfo()

    # Print the available products
    print("Available Sentinel-2 products:")
    print(products_available)

except Exception as e:
    print(f"Error: {e}")

flipped
GeoJSON Structure:
{
  "type": "Polygon",
  "coordinates": [
    [
      [
        120.89150596800005,
        31.61813572400007
      ],
      [
        120.91329390900012,
        30.628070840000078
      ],
      [
        122.0586861060001,
        30.641410783000026
      ],
      [
        122.04885052200007,
        31.632004205000044
      ],
      [
        120.89150596800005,
        31.61813572400007
      ]
    ]
  ]
}
ee.Geometry created successfully.
Available Sentinel-2 products:
['20150825T023846_20150825T023847_T51RVQ', '20150825T023846_20161001T093607_T51RVQ', '20151024T022752_20151024T023354_T51RVQ', '20151024T022752_20161221T062557_T51RVQ', '20151203T023902_20151203T023858_T51RUQ', '20151203T023902_20151203T023858_T51RVQ', '20151203T023902_20170525T083557_T51RUQ', '20151203T023902_20170525T083557_T51RVQ', '20151203T023916_20151203T075500_T51RUQ', '20151203T023916_20151203T075500_T51RVQ', '20151206T024852_20151206T024854_T51RVQ', '20151206T024852_20170519T060

### Create some mosaics of the filtered image collection

In [7]:
# Define the date range and number of intervals
start_date = '2015-08-25'
end_date = '2021-12-29'
num_intervals = 7 # Here the number of collections we want in that time frame

# Convert to datetime objects
start = datetime.datetime.strptime(start_date, '%Y-%m-%d')
end = datetime.datetime.strptime(end_date, '%Y-%m-%d')

# Calculate the total number of days and interval length
total_days = (end - start).days
interval_length = total_days // num_intervals

# Center of the map
center = [centroid.y, centroid.x]

# Create a Folium map
m = folium.Map(location=center, zoom_start=9)

# Loop through each interval and create a mosaic
for i in range(num_intervals):
    interval_start = (start + datetime.timedelta(days=i * interval_length)).strftime('%Y-%m-%d')
    interval_end = (start + datetime.timedelta(days=(i + 1) * interval_length - 1)).strftime('%Y-%m-%d')
    
    # Define the Sentinel-2 image collection for the current interval
    s2_collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
        .filterBounds(ee_geometry) \
        .filterDate(interval_start, interval_end) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) \
        .sort('CLOUDY_PIXEL_PERCENTAGE') \
        .sort('system:time_start', False)
    
    # Add a cloud score band to each image in the collection
    def add_cloud_score(image):
        # Select the QA60 band (cloud mask band) and invert it to create a cloud score
        cloud_mask = image.select('QA60').Not().rename('cloud_score')
        return image.addBands(cloud_mask)
    
    s2_collection_with_clouds = s2_collection.map(add_cloud_score)

    # Create a mosaic using the qualityMosaic method, which prioritizes lower cloud scores
    s2_mosaic_cloud_free = s2_collection_with_clouds.qualityMosaic('cloud_score')
    
    vis_params = {
        'bands': ['B4', 'B3', 'B2'],  # RGB bands
        'min': 0,
        'max': 3000,
        'gamma': 1.4 # to adjust the brightness and contrast: greater than 1 will increase the contrast, while a value less than 1 will decrease it
    }
    
    s2_tile_layer = folium.TileLayer(
        tiles=s2_mosaic_cloud_free.getMapId(vis_params)['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=f'Sentinel-2 Cloud-Free Mosaic {interval_start} to {interval_end}',
        overlay=True,
        control=True
    )
    s2_tile_layer.add_to(m)

# Add the GeoDataFrame as a GeoJSON layer to the map
folium.GeoJson(gdf, name='ROI').add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Display the map
display(m)

### Export procedure still not working

In [3]:
import ee
import folium
import datetime
import os
import urllib.request

# Initialize Earth Engine
ee.Initialize()

# Define the date range and number of intervals
start_date = '2015-08-25'
end_date = '2021-12-29'
num_intervals = 7  # Number of intervals in the time frame

# Define your ROI (Region of Interest) as an example
# Replace with your actual ROI geometry (e.g., from a GeoDataFrame)
ee_geometry = ee.Geometry.Polygon(
    [[[-115.80932797811127, 37.27207677390319],
      [-115.80932797811127, 36.93958011372615],
      [-115.37138722552552, 36.93958011372615],
      [-115.37138722552552, 37.27207677390319]]])

# Convert to datetime objects
start = datetime.datetime.strptime(start_date, '%Y-%m-%d')
end = datetime.datetime.strptime(end_date, '%Y-%m-%d')

# Calculate the total number of days and interval length
total_days = (end - start).days
interval_length = total_days // num_intervals

# Function to clip an image to a region of interest
def clip_image(image, region):
    return image.clip(region)

# Function to export image in tiles
def export_image_in_tiles(image, region, scale, file_prefix, output_dir):
    # Get bounds of the region
    region_bounds = region.bounds().getInfo()['coordinates'][0]
    min_lon, min_lat = region_bounds[0]
    max_lon, max_lat = region_bounds[2]
    
    # Define tile size (e.g., 0.1 degrees ~ 11100 meters at equator)
    tile_size = 0.1
    
    # Calculate number of tiles
    num_tiles_x = int((max_lon - min_lon) / tile_size) + 1
    num_tiles_y = int((max_lat - min_lat) / tile_size) + 1
    
    # Export each tile
    for tile_x in range(num_tiles_x):
        for tile_y in range(num_tiles_y):
            # Define tile bounds
            tile_min_lon = min_lon + tile_x * tile_size
            tile_min_lat = min_lat + tile_y * tile_size
            tile_max_lon = min(tile_min_lon + tile_size, max_lon)
            tile_max_lat = min(tile_min_lat + tile_size, max_lat)
            
            # Create tile geometry
            tile_geometry = ee.Geometry.Rectangle([tile_min_lon, tile_min_lat, tile_max_lon, tile_max_lat])
            
            # Clip image to tile
            clipped_image = clip_image(image, tile_geometry)
            
            # Export tile
            tile_file_name = f'{file_prefix}_tile_{tile_x}_{tile_y}.tif'
            tile_file_path = os.path.join(output_dir, tile_file_name)
            
            try:
                # Export as GeoTIFF with specified parameters
                task = ee.batch.Export.image.toDrive(image=clipped_image,
                                                     description=file_prefix,
                                                     fileNamePrefix=tile_file_name,
                                                     region=tile_geometry,
                                                     scale=scale,
                                                     crs='EPSG:4326')
                task.start()
                print(f'Exporting {tile_file_name}')
                
            except ee.EEException as e:
                print(f'Error exporting {tile_file_name}: {str(e)}')

# Loop through each interval and create a mosaic
for i in range(num_intervals):
    interval_start = (start + datetime.timedelta(days=i * interval_length)).strftime('%Y-%m-%d')
    interval_end = (start + datetime.timedelta(days=(i + 1) * interval_length - 1)).strftime('%Y-%m-%d')
    
    # Define the Sentinel-2 image collection for the current interval
    s2_collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
        .filterBounds(ee_geometry) \
        .filterDate(interval_start, interval_end) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) \
        .sort('CLOUDY_PIXEL_PERCENTAGE') \
        .sort('system:time_start', False)
    
    # Add a cloud score band to each image in the collection
    def add_cloud_score(image):
        cloud_mask = image.select('QA60').Not().rename('cloud_score')
        return image.addBands(cloud_mask)
    
    s2_collection_with_clouds = s2_collection.map(add_cloud_score)

    # Create a mosaic using the qualityMosaic method
    s2_mosaic_cloud_free = s2_collection_with_clouds.qualityMosaic('cloud_score')
    
    # Clip mosaic to ROI
    s2_mosaic_clipped = clip_image(s2_mosaic_cloud_free, ee_geometry)

    # Define a directory to save locally
    output_dir = r'C:\Users\andre\OneDrive - Politecnico di Milano\Earth_observation\project\Mod_b\mosaics'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Export the clipped mosaic in tiles
    file_prefix = f'Sentinel2_Mosaic_{interval_start}_{interval_end}'
    export_image_in_tiles(s2_mosaic_clipped, ee_geometry, scale=10, file_prefix=file_prefix, output_dir=output_dir)

# Print message when done
print('Export process complete.')


Error exporting Sentinel2_Mosaic_2015-08-25_2016-07-20_tile_0_0.tif: Request had insufficient authentication scopes.
Error exporting Sentinel2_Mosaic_2015-08-25_2016-07-20_tile_0_1.tif: Request had insufficient authentication scopes.
Error exporting Sentinel2_Mosaic_2015-08-25_2016-07-20_tile_0_2.tif: Request had insufficient authentication scopes.
Error exporting Sentinel2_Mosaic_2015-08-25_2016-07-20_tile_0_3.tif: Request had insufficient authentication scopes.
Error exporting Sentinel2_Mosaic_2015-08-25_2016-07-20_tile_1_0.tif: Request had insufficient authentication scopes.
Error exporting Sentinel2_Mosaic_2015-08-25_2016-07-20_tile_1_1.tif: Request had insufficient authentication scopes.
Error exporting Sentinel2_Mosaic_2015-08-25_2016-07-20_tile_1_2.tif: Request had insufficient authentication scopes.
Error exporting Sentinel2_Mosaic_2015-08-25_2016-07-20_tile_1_3.tif: Request had insufficient authentication scopes.
Error exporting Sentinel2_Mosaic_2015-08-25_2016-07-20_tile_2_0.

Error exporting Sentinel2_Mosaic_2018-05-14_2019-04-09_tile_2_3.tif: Request had insufficient authentication scopes.
Error exporting Sentinel2_Mosaic_2018-05-14_2019-04-09_tile_3_0.tif: Request had insufficient authentication scopes.
Error exporting Sentinel2_Mosaic_2018-05-14_2019-04-09_tile_3_1.tif: Request had insufficient authentication scopes.
Error exporting Sentinel2_Mosaic_2018-05-14_2019-04-09_tile_3_2.tif: Request had insufficient authentication scopes.
Error exporting Sentinel2_Mosaic_2018-05-14_2019-04-09_tile_3_3.tif: Request had insufficient authentication scopes.
Error exporting Sentinel2_Mosaic_2018-05-14_2019-04-09_tile_4_0.tif: Request had insufficient authentication scopes.
Error exporting Sentinel2_Mosaic_2018-05-14_2019-04-09_tile_4_1.tif: Request had insufficient authentication scopes.
Error exporting Sentinel2_Mosaic_2018-05-14_2019-04-09_tile_4_2.tif: Request had insufficient authentication scopes.
Error exporting Sentinel2_Mosaic_2018-05-14_2019-04-09_tile_4_3.